In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
from mplsoccer.pitch import Pitch
from mplsoccer import Sbopen
import json

In [2]:
# replace this with a valid path on your local machine
# load configuration from file
with open('../config.json', 'r') as config_file:
    config = json.load(config_file)

db_path = config.get('statsbomb_db_path') # get the relevant db path from config

# create sqlalchemy engine to connect to the SQLite database
engine = create_engine(f'sqlite:///{db_path}')

# instantiate a parser object
parser = Sbopen()


In [3]:
# retrieve competitions
df_competition = parser.competition()
# write the competitions to the database, statsbomb.db
data = df_competition

df = pd.DataFrame(data)

# use the to_sql function to write the DataFrame to the SQLite database
#df.to_sql('competition', engine, if_exists='replace', index=False)


67

In [4]:
df.head()

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-08-17T23:51:11.837478,None,None,2023-08-17T23:51:11.837478
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00


In [5]:
matches = pd.DataFrame()
for i, comp in df_competition.iterrows():
    print("Entered:",i)
    if i == 0:
        matches = parser.match(competition_id=comp.competition_id, season_id=comp.season_id)
    else:
        stg = parser.match(competition_id=comp.competition_id, season_id=comp.season_id)
        matches = pd.concat([matches, stg], axis=0, ignore_index=True)

Entered: 0
Entered: 1
Entered: 2
Entered: 3
Entered: 4
Entered: 5
Entered: 6
Entered: 7
Entered: 8
Entered: 9
Entered: 10
Entered: 11
Entered: 12
Entered: 13
Entered: 14
Entered: 15
Entered: 16
Entered: 17
Entered: 18
Entered: 19
Entered: 20
Entered: 21
Entered: 22
Entered: 23
Entered: 24
Entered: 25
Entered: 26
Entered: 27
Entered: 28
Entered: 29
Entered: 30
Entered: 31
Entered: 32
Entered: 33
Entered: 34
Entered: 35
Entered: 36
Entered: 37
Entered: 38
Entered: 39
Entered: 40
Entered: 41
Entered: 42
Entered: 43
Entered: 44
Entered: 45
Entered: 46
Entered: 47
Entered: 48
Entered: 49
Entered: 50
Entered: 51
Entered: 52
Entered: 53
Entered: 54
Entered: 55
Entered: 56
Entered: 57
Entered: 58
Entered: 59
Entered: 60
Entered: 61
Entered: 62
Entered: 63
Entered: 64
Entered: 65
Entered: 66


In [5]:
conn = sqlite3.connect(db_path)
matches = pd.read_sql_query(
    '''SELECT * 
    FROM match;
    ''', conn)
conn.close()
match_list = matches.match_id.unique()

In [6]:
%%time 
related_df_list = []
freeze_df_list = []
tactics_df_list = []

for i, match in enumerate(match_list):
    print(f"{i} - ENTERED :",match)
    #related_df_list.append(parser.event(match)[1])
    #print("related appended\n---------------------")
    freeze_df_list.append(parser.event(match)[2])
    print("freeze appended\n---------------------")
    tactics_df_list.append(parser.event(match)[3])
    print("tactics appended\n---------------------")
print("appending complete")

#df_related = pd.concat(related_df_list)
#print("related concatenated")
df_freeze = pd.concat(freeze_df_list)
print("freeze concatenated")
df_tactics = pd.concat(tactics_df_list)
print("tactics concatenated")
print("Concatenation done")

# in order not to do the same data retrieval over and over, use the to_sql function to write the dataframe to the sqlite database
#df_related.to_sql('related', engine, if_exists='replace', index=False)
#print("related inserted")
df_freeze.to_sql('freeze', engine, if_exists='replace', index=False)
print("freeze inserted")
df_tactics.to_sql('tactics', engine, if_exists='replace', index=False)
print("tactics inserted")

0 - ENTERED : 3890561
freeze appended
---------------------
tactics appended
---------------------
1 - ENTERED : 3890505
freeze appended
---------------------
tactics appended
---------------------
2 - ENTERED : 3890511
freeze appended
---------------------
tactics appended
---------------------
3 - ENTERED : 3890515
freeze appended
---------------------
tactics appended
---------------------
4 - ENTERED : 3890411
freeze appended
---------------------
tactics appended
---------------------
5 - ENTERED : 3890397
freeze appended
---------------------
tactics appended
---------------------
6 - ENTERED : 3890401
freeze appended
---------------------
tactics appended
---------------------
7 - ENTERED : 3890396
freeze appended
---------------------
tactics appended
---------------------
8 - ENTERED : 3890384
freeze appended
---------------------
tactics appended
---------------------
9 - ENTERED : 3890385
freeze appended
---------------------
tactics appended
---------------------
10 - ENTER

In [8]:
# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Retrieve all matchdata
query = '''
SELECT * FROM match 
WHERE home_team_gender = "male"
'''

# Use the connection and query to create a DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Print the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2690 entries, 0 to 2689
Data columns (total 52 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   match_id                         2690 non-null   int64  
 1   match_date                       2690 non-null   object 
 2   kick_off                         2686 non-null   object 
 3   home_score                       2690 non-null   int64  
 4   away_score                       2690 non-null   int64  
 5   match_status                     2690 non-null   object 
 6   match_status_360                 2690 non-null   object 
 7   last_updated                     2690 non-null   object 
 8   last_updated_360                 1138 non-null   object 
 9   match_week                       2690 non-null   int64  
 10  competition_id                   2690 non-null   int64  
 11  country_name                     2690 non-null   object 
 12  competition_name    

In [9]:
%%time
#matches = df.copy()
lineups = []

for i, match in matches.iterrows():
    print(f"{i} - ENTERED :",match.match_id)
    lineups.append(parser.lineup(match.match_id))
    print("lineup appended\n---------------------")
print("appending complete")

lineup = pd.concat(lineups)
print("Concatenation done")

0 - ENTERED : 3890561
lineup appended
---------------------
1 - ENTERED : 3890505
lineup appended
---------------------
2 - ENTERED : 3890511
lineup appended
---------------------
3 - ENTERED : 3890515
lineup appended
---------------------
4 - ENTERED : 3890411
lineup appended
---------------------
5 - ENTERED : 3890397
lineup appended
---------------------
6 - ENTERED : 3890401
lineup appended
---------------------
7 - ENTERED : 3890396
lineup appended
---------------------
8 - ENTERED : 3890384
lineup appended
---------------------
9 - ENTERED : 3890385
lineup appended
---------------------
10 - ENTERED : 3890282
lineup appended
---------------------
11 - ENTERED : 3890287
lineup appended
---------------------
12 - ENTERED : 3890269
lineup appended
---------------------
13 - ENTERED : 3890268
lineup appended
---------------------
14 - ENTERED : 3890402
lineup appended
---------------------
15 - ENTERED : 3890564
lineup appended
---------------------
16 - ENTERED : 3890563
lineup appe

In [7]:
# write the matches to the database as a new table, statsbomb.db / match
data = matches

# Create an SQLAlchemy engine to connect to the SQLite database
engine = create_engine(f'sqlite:///{db_path}')

# Use the to_sql function to write the DataFrame to the SQLite database
data.to_sql('match', engine, if_exists='replace', index=False)

3199

In [10]:
lineup = pd.concat(lineups)
print("Concatenation done")

Concatenation done


In [11]:
# write the matches to the database as a new table, statsbomb.db / match
data = lineup

# Use the to_sql function to write the DataFrame to the SQLite database
data.to_sql('lineup', engine, if_exists='replace', index=False)

119984

In [12]:
conn = sqlite3.connect(db_path)
pd.read_sql_query(
    '''SELECT * 
    FROM lineup;
    ''', conn)
conn.close()

,player_id,player_name,player_nickname,jersey_number,match_id,team_id,team_name,country_id,country_name
0,3053,Leroy Sané,Leroy Sané,19,3890561,181,Schalke 04,85.0,Germany
1,3499,Jean-Eric Maxim Choupo-Moting,Eric Maxim Choupo-Moting,13,3890561,181,Schalke 04,39.0,Cameroon
2,3502,Joël Andre Job Matip,Joël Matip,32,3890561,181,Schalke 04,39.0,Cameroon
3,3510,Sead Kolašinac,Sead Kolašinac,6,3890561,181,Schalke 04,28.0,Bosnia and Herzegovina
4,3570,Pierre-Emile Højbjerg,Pierre-Emile Højbjerg,23,3890561,181,Schalke 04,61.0,Denmark
...,...,...,...,...,...,...,...,...,...
119979,25840,Celia Jiménez Delgado,Celia Jiménez,2,69161,863,Spain Women's,214.0,Spain
119980,25842,Andrea Sánchez Falcón,Andrea Falcón,21,69161,863,Spain Women's,214.0,Spain
119981,45667,María Dolores Gallardo Nuñez,Lola Gallardo,1,69161,863,Spain Women's,214.0,Spain
119982,49831,Ivana Andrés Sanz,Ivana Andrés,5,69161,863,Spain Women's,214.0,Spain
